In [56]:
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists

### Definition of reference potential

In [57]:
class potential:
    def __init__(self, x, y, q):
        self.point_x = x
        self.point_y = y
        self.point_charge = q

### Object for many potentials

In [58]:
class MUL_potential:
    def __init__(self, x_arr: np.array([]), y_arr: np.array([]), charge_arr: np.array([])):
        if(len(x_arr) != len(y_arr) or len(x_arr) != len(charge_arr)):
            print("Can't initialize using arrays of different length.")

        else:
            self.potentials = np.array([])

            for i in range(0, len(x_arr)):
                self.potentials = np.append(self.potentials, potential(x_arr[i], y_arr[i], charge_arr[i]))

### Definition of particle

In [59]:
class particle:
    def __init__(self, alpha, d, q):
        self.particle_angle = alpha
        self.particle_len = d
        self.particle_charge = q

        #first electron
        self.e1_x = self.particle_len * math.cos((math.pi / 180) * self.particle_angle)
        self.e1_y = self.particle_len * math.sin((math.pi / 180) * self.particle_angle)

        #2nd electron
        self.e2_x = self.particle_len * math.cos((math.pi / 180) * (self.particle_angle + 120))
        self.e2_y = self.particle_len * math.sin((math.pi / 180) * (self.particle_angle + 120))

        #3rd electron
        self.e3_x = self.particle_len * math.cos((math.pi / 180) * (self.particle_angle + 240))
        self.e3_y = self.particle_len * math.sin((math.pi / 180) * (self.particle_angle + 240))

### Functions for potential and energy for single reference potential

In [60]:
def getPotential(p: particle, r: potential):
    v1 = 1.0 / (math.sqrt(math.pow(r.point_x - p.e1_x, 2) + math.pow(r.point_y - p.e1_y, 2)))
    v2 = 1.0 / (math.sqrt(math.pow(r.point_x - p.e2_x, 2) + math.pow(r.point_y - p.e2_y, 2)))
    v3 = 1.0 / (math.sqrt(math.pow(r.point_x - p.e3_x, 2) + math.pow(r.point_y - p.e3_y, 2)))

    return v1 + v2 + v3

def getEnergy(p: particle, r: potential):
    k_c = 8.9875517923 * 1.60217662 / 10

    return k_c * r.point_charge * p.particle_charge * getPotential(p, r)

### Energy for multiple potentials

In [61]:
def getEnergy_MUL(p: particle, pot: MUL_potential):
    k_c = 8.9875517923 * 1.60217662 / 10
    acc = 0

    for i in range(0, len(pot.potentials)):
        r = pot.potentials[i]
        acc += k_c * r.point_charge * p.particle_charge * getPotential(p, r)

    return acc

### Function to rotate particle with single potential

In [62]:
def tryRotating(p: particle, r: potential, alpha, T):
    #create particle at new angle
    p1 = particle(p.particle_angle + alpha, p.particle_len, p.particle_charge)

    #get old and new energy
    E_old = getEnergy(p, r)
    E_new = getEnergy(p1, r)

    #boltzmann constant in [eV/K]
    k_b = 8.617333262145 * math.pow(10, -5)

    #if new energy is lower than old, return new angle
    if(E_new - E_old < 0):
        return p1

    else:
        w = math.exp(-(E_new - E_old) / (T * k_b))
        rand = random.random()

        if(rand <= w):
            return p1
        else:
            return p

### Rotating for multiple potentials

In [63]:
def tryRotating_MUL(p: particle, pot: MUL_potential, alpha, T):
    #create particle at new angle
    p1 = particle(p.particle_angle + alpha, p.particle_len, p.particle_charge)

    #get old and new energy
    E_old = getEnergy_MUL(p, pot)
    E_new = getEnergy_MUL(p1, pot)

    #boltzmann constant in [eV/K]
    k_b = 8.617333262145 * math.pow(10, -5)

    #if new energy is lower than old, return new angle
    if(E_new - E_old < 0):
        return p1

    else:
        w = math.exp(-(E_new - E_old) / (T * k_b))
        rand = random.random()

        if(rand <= w):
            return p1
        else:
            return p

### Order parameter

In [64]:
def order_ite(p: particle):
    cos_ite = round(math.cos((math.pi / 180) * (3 * p.particle_angle)), 5)
    sin_ite = round(math.sin((math.pi / 180) * (3 * p.particle_angle)), 5)

    return cos_ite, sin_ite

def order_full(cos, sin, steps):
    thet = 0.0

    #get mean
    cos = cos / steps
    sin = sin / steps

    #get theta
    thet = math.atan(sin / cos)

    #cube them
    cos = math.pow(cos, 2)
    sin = math.pow(sin, 2)

    return cos + sin, thet

### Fluctuations

In [65]:
def fluct(arr_x: np.array, arr_y: np.array, delta):
    for i in range(len(arr_x)):
        k = True

        while(k == True):
            x = delta * (2 * random.random() - 1.0)
            y = delta * (2 * random.random() - 1.0)

            if(x**2 + y**2 <= delta):
                k = False
        
        arr_x[i] += x
        arr_y[i] += y

    return arr_x, arr_y

### Physical constants and variables for simulation

In [66]:
#physical stuff - charge is in [e], distances are in [nm]
phi = 0.5
T = 5 * pow(10, -3)

#multipliers for particle
d1 = 0.2
Q2 = 0.5
Q1 = -300
pot_q = np.array([Q1, Q1, Q1])

AngleInt = int(60 / phi)

#MC steps
mc = 2000000

### Main loop to generate order parameters

In [67]:
d2 = [30, 60, 90]
delta = [1, 5, 10]

#array to keep order parameters
vals = np.array([])

#generate particle
p = particle(0, d1, Q2)

#loop for base distances
for i in d2:
    print()
    print('Currently running R = ' + str(i))

    #generate base distances of point sources
    pot_x = np.array([i, i * math.cos((math.pi / 180) * 120), i * math.cos((math.pi / 180) * 240)])
    pot_y = np.array([0, i * math.sin((math.pi / 180) * 120), i * math.sin((math.pi / 180) * 240)])

    #loop for fluctuations
    for j in delta:
        print()
        print('Delta = ' + str(j))

        #loop to run the same variables a few times
        for k in range(5):
            #introduce fluctuations to starting positions
            fluct_x, fluct_y = fluct(pot_x, pot_y, j)   

            #generate point sources
            ref = MUL_potential(fluct_x, fluct_y, pot_q)

            #order parameter
            cos = 0
            sin = 0

            for n in range(0, mc):

                #generate random number to check which way we're gonna rotate
                rng = random.random()

                #rotate counter-clockwise
                if(rng > 0.5):
                    p = tryRotating_MUL(p, ref, phi, T)
                #rotate clockwise
                else:
                    p = tryRotating_MUL(p, ref, -phi, T)

                #calculate cos and sin for order parameter
                cos_ite, sin_ite = order_ite(p)
                cos += cos_ite
                sin += sin_ite 

            #order parameter and theta
            order, theta = order_full(cos, sin, mc)

            #append values to array
            vals = np.append(vals, [i, j, order, theta])

            print('Order parameter = ' + str(order))


Currently running R = 30

Delta = 1
Order parameter = 0.9649036664909579
Order parameter = 0.9680288777873279
Order parameter = 0.968420384083408
Order parameter = 0.9693030984721502
Order parameter = 0.9686612838656123

Delta = 5
Order parameter = 0.9748815743355184
Order parameter = 0.9771635748048126
Order parameter = 0.9742791104544035
Order parameter = 0.975640112441051
Order parameter = 0.9785649709337162

Delta = 10
Order parameter = 0.9742194384428959
Order parameter = 0.9775505403530562
Order parameter = 0.9795598973868871
Order parameter = 0.9814380034075542
Order parameter = 0.9818402927172475

Currently running R = 60

Delta = 1
Order parameter = 0.45184026394608123
Order parameter = 0.4448880846183133
Order parameter = 0.4369777155594872
Order parameter = 0.4120341603554642
Order parameter = 0.452487048135191

Delta = 5
Order parameter = 0.43582762469115494
Order parameter = 0.4177859270782327
Order parameter = 0.4176904711794308
Order parameter = 0.42781394629928304
Orde

### Opening file to append

In [78]:
r_loop = np.array([])
delta_loop = np.array([])
order_loop = np.array([])
theta_loop = np.array([])


for i in range(int(len(vals) / 4)):
    r_loop = np.append(r_loop, vals[4 * i])
    delta_loop = np.append(delta_loop, vals[(4 * i) + 1])
    order_loop = np.append(order_loop, vals[(4 * i) + 2])
    theta_loop = np.append(theta_loop, vals[(4 * i) + 3])

45

In [80]:
vals = pd.DataFrame(r_loop, columns = ['R'])
vals['Delta'] = delta_loop
vals['Order'] = order_loop
vals['Theta'] = theta_loop
pd.DataFrame.to_csv(vals, 'Results\\Fluctuations.csv')